# --- Day 11: Reactor ---


In [7]:
# --- Support Functions ---
# This section contains support functions used by the main code.
!pip install -q tqdm

from tqdm.notebook import tqdm
import time

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()
    
def traverse_tree(start: str, end: str, tree: dict) -> list[list]:
    paths=[]
    queue = [[start]]
    pbar = tqdm(desc="Tree Exploration", dynamic_ncols=True)
    idx = 0
    while queue:
        path = queue.pop()
        idx +=1    
        pbar.set_description(f"Visito: {idx}: {path[-1]} | Trovate: {len(paths)}")
        for b in tree.get(path[-1], []):
            p = path.copy()+[b]
            if b == end:
                paths.append(p)
                pbar.update(1)
                pbar.set_description(f"Trovate: {len(paths)}")
                break
            queue.append(p)
    pbar.close()
    return(paths)

def rev_tree(tree: dict) -> dict:
    rtree = {}
    for p, cs in tree.items():
        for c in cs:
            rtree[c] = rtree.get(c, []) + [p]
    return rtree

def can_reach(target : str, tree: dict) -> set:
    cr = set() | {target}
    queue = [target]
    while queue:
        node = queue.pop(0)
        ns = tree.get(node, [])
        if not ns:
            continue
        # print (f"node: {node}, queue = {queue}, cr = {cr}")
        for n in ns:
            if n in cr:
                continue
            cr.add(n)
            queue.append(n)
    return cr

def traverse_pruned(
    start: str, end: str, tree: dict[str, list[str]], 
    can_fft: set[str], can_dac: set[str], can_out: set[str]) -> list[list[str]]:
    results=[]
    stack = [(start, [start], {start}, start == 'fft', start == 'dac')] # node, [path], {seen}

    pbar = tqdm(desc="Tree Exploration", dynamic_ncols=True)
    visited = 0

    while stack:
        node, path, seen, has_fft, has_dac = stack.pop(0)

        visited +=1    
        pbar.set_description(f"Visit: {visited}: {node} | Found: {len(results)}")

        # global pruning
        if node not in can_out:
            continue
        if not has_fft and node not in can_fft:
            continue
        if not has_dac and node not in can_dac:
            continue

        for child in tree.get(node, []):
            if child in seen:
                continue
            n_fft = has_fft or child == 'fft'
            n_dac = has_dac or child == 'dac'
            if child == end:
                if n_fft and n_dac:
                    results.append(path + [child])
                    pbar.update(1)
                    pbar.set_description(f"Trovate: {len(results)}")
                continue
            
            stack.append((child, path, seen | {child}, n_fft, n_dac ))

    pbar.close()
    return results

def count_paths(start, end, tree):
    rev = rev_tree(tree)

    can_fft = can_reach('fft', rev)
    can_dac = can_reach('dac', rev)
    can_out = can_reach(end, rev)

    seen_global = set() | {start}
    
    def dfs(node, seen, has_fft, has_dac):
        if node not in can_out:
            return 0
        if not has_fft and node not in can_fft:
            return 0
        if not has_dac and node not in can_dac:
            return 0

        total = 0
        if node == end:
            ret = int(has_fft and has_dac)
            if ret:
                print("Found: ",total + 1, "   ", end="\r")
            return ret
        for nxt in tree.get(node, []):
            #if nxt in seen_global:
            #    continue
            if nxt in seen:
                continue
            # seen_global.add(nxt)
            # print (len(seen_global))
            # if total == 1: #debug
            #     break
            total += dfs(
                nxt,
                seen | {nxt},
                has_fft or nxt == 'fft',
                has_dac or nxt == 'dac'
            )
        return total

    #return dfs(start, {start}, start == 'fft', start == 'dac')


In [ ]:
# --- Part ONE ---

input = read_input('input.txt')

start = 'you'
end = 'out'

# construct "tree" of parents and childs
childs = {}
parents = {}  # not necessary
for l in input:
    p, cs = l.split(':')
    cs = cs.strip().split(' ')
    childs[p] = cs
    for c in cs:
        pp = (parents.get(c, []))
        pp.append(p)
        parents[c] = pp
# print(f"{childs}")
# print(f"{parents}")

paths = traverse_tree(start, end, childs)

result = len(paths)

print("Part ONE:", result)

In [8]:
# --- Part TWO ---
input = read_input('test2.txt')

result = 0

start = 'svr'
end = 'out'

# construct "tree" of parents and childs
childs = {}
parents = {} # not necessary
for l in input:
    p, cs = l.split(':')
    cs = cs.strip().split(' ')
    childs[p] = cs
    for c in cs:
        pp = (parents.get(c, []))
        pp.append(p)
        parents[c] = pp
# print(childs)

rtree = rev_tree(childs)
#for k, v in rtree.items():
#    print (f"{k}: {' '.join(v)}")

can_reach_out = can_reach('out', rtree)
# print ("can_reach_out", can_reach_out)
can_reach_dac = can_reach('dac', rtree)
# print ("can_reach_dac", can_reach_dac)
can_reach_fft = can_reach('fft', rtree)
# print ("can_reach_fft", can_reach_fft)

# paths = traverse_pruned(start, end, childs, can_reach_fft, can_reach_dac, can_reach_out)
#paths = traverse_tree(end, start, rtree)

result = count_paths(end, start, rtree)

# for p in paths:
#    if ('dac' in p) and ('fft' in p):
#        result += 1
#    print (p)

#


print("Part TWO:", result)

Part TWO: None
